In [1]:
import numpy as np
import pandas as pd
import random as rd
from collections import defaultdict
%pylab inline

Populating the interactive namespace from numpy and matplotlib


## Load Data 

In [2]:
train = pd.read_csv('data/train.csv')
train = train.set_index('ID')

/Users/zach/anaconda/lib/python2.7/site-packages/pandas/io/parsers.py:1170: DtypeWarning: Columns (8,9,10,11,12,43,157,196,214,225,228,229,231,235,238) have mixed types. Specify dtype option on import or set low_memory=False.
  data = self._reader.read(nrows)


Get a sample to play with if we need it

In [3]:
samp = train.loc[rd.sample(train.index, 5000)]

## EDA

In [4]:
# Variable types
vartypes = train.dtypes

# Divide into categories
cat_df = train.loc[:, vartypes[vartypes=='O'].index]
int_df = train.loc[:, vartypes[vartypes=='int64'].index]
float_df = train.loc[:, vartypes[vartypes=='float64'].index]


### Categorical

The first thing we need to do is make a summary of all the categorical variables, so that we can figure out how to process them. Things I would like to look at:
- Number of unique values
- Frequency of top value
- Top Item
- Number of missing values
- First few examples
- Spread of target frequencies

In [6]:
from proc_functions import get_summary

# Replace nans
cat_df = cat_df.replace(np.nan, 'NA')

# Summarise each categorical variable
cat_df.apply(lambda col: get_summary(col, train['target'])).T

,Examples,Fraction Missing,Freq Range,NumUnique,Top Freq,Top Item
VAR_0001,"H, R, Q",0,0.06985867,3,0.5845377,R
VAR_0005,"C, B, N, S",0,0.117376,4,0.491968,B
VAR_0008,"False, NA",0,0.000404069,2,0.9996144,False
VAR_0009,"False, NA",0,0.000404069,2,0.9996144,False
VAR_0010,"False, NA",0,0.000404069,2,0.9996144,False
VAR_0011,"False, NA",0,0.000404069,2,0.9996144,False
VAR_0012,"False, NA",0,0.000404069,2,0.9996144,False
VAR_0043,"False, NA",0,0.000404069,2,0.9996144,False
VAR_0044,"[], NA",0,0.000404069,2,0.9996144,[]
VAR_0073,"NA, 04SEP12:00:00:00, 26JAN12:00:00:00, 18SEP1...",0,1,1459,0.6963183,NA


Use the above summaries to categorize the variables in the first part of process_categorical.py

#### Dates
Start by converting all dates to the proper format

In [24]:
from process_categorical import convert_all_date_columns

date_df = convert_all_date_columns(train)
date_df.describe().T

,count,unique,top,freq,first,last
VAR_0073,44104,1458,2009-03-13 00:00:00,260,2008-01-02 00:00:00,2012-10-31 00:00:00
VAR_0075,145175,2371,2010-09-22 00:00:00,1168,2001-01-01 00:00:00,2012-11-01 00:00:00
VAR_0156,5870,730,2011-12-12 00:00:00,45,2008-04-11 00:00:00,2012-10-29 00:00:00
VAR_0157,920,424,2012-07-27 00:00:00,8,2008-10-17 00:00:00,2012-10-31 00:00:00
VAR_0158,2089,407,2012-06-01 00:00:00,28,2008-09-30 00:00:00,2012-10-29 00:00:00
VAR_0159,5870,650,2012-06-04 00:00:00,49,2008-09-23 00:00:00,2012-10-29 00:00:00
VAR_0166,14230,2145,2011-12-19 00:00:00,45,2002-07-30 00:00:00,2012-10-30 00:00:00
VAR_0167,2567,853,2011-11-03 00:00:00,27,2005-02-12 00:00:00,2012-10-30 00:00:00
VAR_0168,10725,1645,2011-11-03 00:00:00,206,1999-12-31 00:00:00,2012-11-01 00:00:00
VAR_0169,14230,1908,2011-12-19 00:00:00,48,2002-07-30 00:00:00,2012-11-01 00:00:00


Now do a little bit of feature engineering on dates
- Take all pairwise differences between dates
- Earliest date record
- Most recent date record
- Number of non-nan date records

In [25]:
# Feature engineering on dates

### Numeric Variables
I think there are not any actual float variables (non-integer) in the data. It looks like features with negative entries (just -1?) are being characterized as floats.

We should verify this, but then everything can be treated the same.

In [26]:
# Verify difference between integer and float

#### Integer
 
- Each integer valued variable needs to be checked to see if it should be treated as categorical or numerical
- It's possible these can all be taken care of, just by using trees
- If mutual information

#### Are all the missing values (99, 999, etc..) in the same spot?

#### Float
- Float variables _should_ be the easiest---they can be taken at face value

Combine models from all three variable types